<a href="https://colab.research.google.com/github/OzasaHiro/git-practice/blob/main/birdclef2022_audio_to_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
IS_COLAB = not os.path.exists('/kaggle/input')
print(IS_COLAB) 

True


In [2]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  from kaggle_datasets import KaggleDatasets

Mounted at /content/drive


In [6]:
!pip install /content/drive/MyDrive/Kaggle-BirdCLEF2022/torchlibrosa-0.0.5-py3-none-any.whl > /dev/null

In [7]:
import os, json, random, cv2
import audioread
import logging
import gc
import sys
sys.path.append('/content/drive/MyDrive/Kaggle-BirdCLEF2022/pytorch-image-models-master')
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
from sklearn.model_selection import StratifiedKFold, GroupKFold

from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm

import albumentations as A
import albumentations.pytorch.transforms as T

import matplotlib.pyplot as plt

In [12]:
%%time

### Create Kaggle Dataset if not exists 

DATASET_NAME = f'birdclef2022-audio-5to10sec'

os.makedirs(f'/content/drive/MyDrive/Kaggle-BirdCLEF2022/{DATASET_NAME}', exist_ok=True)


CPU times: user 1.06 ms, sys: 4 µs, total: 1.07 ms
Wall time: 25.5 ms


In [13]:
SR = 32000
USE_SEC = 5 # 60 # 90 # 60

def Audio_to_Array(path):
    y, sr = sf.read(path, always_2d=True)
    y = np.mean(y, 1) # there is (X, 2) array

    END = 2 * SR * USE_SEC
    START = SR * USE_SEC

   # if len(y) > (USE_SEC + 2) * SR:
   #     y = y[SR:-SR]

    if len(y) > 2 * SR * USE_SEC:
        y = y[START : END]

    return y

def save_(path):
    save_path = f'/content/drive/MyDrive/Kaggle-BirdCLEF2022/{DATASET_NAME}/train_np/' + "/".join(path.split('/')[-2:])
    np.save(save_path, Audio_to_Array(path))


AUDIO_PATH = '/content/drive/MyDrive/Kaggle-BirdCLEF2022/train_audio'


train = pd.read_csv('/content/drive/MyDrive/Kaggle-BirdCLEF2022/train_metadata.csv')
train["file_path"] = AUDIO_PATH + '/' + train['filename']
paths = train["file_path"].values

In [14]:
i = 0 # 0,1,2,3
step = int(len(paths)/1)
# paths[step * i:step * (i+1)]

In [15]:
NUM_WORKERS = 4
CLASSES = sorted(os.listdir(AUDIO_PATH))

for dir_ in tqdm(CLASSES):
    _ = os.makedirs(f'/content/drive/MyDrive/Kaggle-BirdCLEF2022/{DATASET_NAME}/train_np/' + dir_, exist_ok=True)
_ = Parallel(n_jobs=NUM_WORKERS)(delayed(save_)(AUDIO_PATH) for AUDIO_PATH in tqdm(paths[step * i:step * (i+1)]))

 10%|▉         | 1472/14852 [01:51<16:54, 13.18it/s]/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 14852/14852 [30:41<00:00,  8.06it/s]
